# Segmentation python notebook

This python notebook is a abridged version of the tutorial script by Henschel et. al. on the usage of Fastsurfer for Segmentation only. The sourcecodes can be found at https://github.com/deep-mi/FastSurfer, and detailed documentation for FastSurfer can be found at https://deep-mi.org/FastSurfer/dev/index.html. When complete, the segmentation would be downloaded to the computer.

---

## Click 1: Setup FastSurfer

In [ ]:
#@title Here we first setup the environment by downloading the open source deep-mi/fastsurfer project and the required packages
import os
import sys
from os.path import exists, join, basename, splitext

print("Starting setup. This could take a few minutes")
print("----------------------------------------------")

is_google_colab = "colab.research.google.com" in str(os.environ)
if is_google_colab:
    # this is for a Google Colab Notebook
    SETUP_DIR = "/content/"
else:
    SETUP_DIR = os.environ["HOME"] + "/fastsurfer_tutorial/"

# Go to the FastSurfer directory
!mkdir -p "{SETUP_DIR}"
%cd "{SETUP_DIR}"

print(f"Using {SETUP_DIR} to store files.")

print("Downloading FastSurfer")
print("----------------------------------------------")


git_repo_url = 'https://github.com/deep-mi/fastsurfer.git'
project_name = splitext(basename(git_repo_url))[0]
FASTSURFER_HOME = SETUP_DIR + project_name + "/"
if not exists(project_name):
  # clone and install dependencies
  ! git clone -q --branch stable $git_repo_url
  ! pip install -r $FASTSURFER_HOME/requirements.txt
sys.path.append(FASTSURFER_HOME)

# Update dependencies
print("Installing required packages")
print("----------------------------------------------")

! pip install torchio==0.18.83
! pip install yacs==0.1.8
! pip install plotly==5.9.0

print("Finished setup")
print("----------------------------------------------")


---

## Click 2: Upload your image

In [ ]:
#@title Click the run button to upload your T1-weighted MRI image (will be deleted after the session is over; make sure you have the permissions to do this for the image in question).
%cd "{SETUP_DIR}"

from google.colab import files
uploaded = files.upload()

img = SETUP_DIR + list(uploaded.keys())[0]

%ls

/content


Saving subject_61_t1wWarped.nii.gz to subject_61_t1wWarped.nii.gz
fastsurfer/  fastsurfer_seg/  sample_data/  subject_61_t1wWarped.nii.gz


---

## Click 3: Run FastSurfer

In [ ]:

#@title The first part of FastSurfer creates a whole-brain segmentation into 95 classes. Here, we use the pretrained deep-learning network FastSurferCNN using the checkpoints stored at the open source project deep-mi/fastsurfer to to run the model inference on a single image.

# Note, you should also add --3T, if you are processing data from a 3T scanner.
! FASTSURFER_HOME=$FASTSURFER_HOME \
  $FASTSURFER_HOME/run_fastsurfer.sh --t1 $img \
                                     --sd "{SETUP_DIR}fastsurfer_seg" \
                                     --sid result \
                                     --vox_size 1\
                                     --seg_only --py python3 \
                                     --parallel --threads 8 \
                                     --allow_root

---

## Click 4: Download the image

In [ ]:
#@title Click this run button, if you would prefer to download the segmentation in nifti-format
import nibabel as nib
# conversion to nifti
data = nib.load(f'{SETUP_DIR}fastsurfer_seg/result/mri/aparc.DKTatlas+aseg.deep.mgz')
img_nifti = nib.Nifti1Image(data.get_fdata(), data.affine, header=nib.Nifti1Header())
nib.nifti1.save(img_nifti, f'{SETUP_DIR}fastsurfer_seg/result/mri/aparc.DKTatlas+aseg.deep.nii.gz')

from google.colab import files
files.download(f'{SETUP_DIR}fastsurfer_seg/result/mri/aparc.DKTatlas+aseg.deep.nii.gz')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---